# Mask detection with mobilenetv2

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer 
import tensorflow as tf
from tensorflow.keras.models import Model ,Sequential 
from tensorflow.keras.layers import Dropout, Dense, Flatten, AveragePooling2D, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img , img_to_array
import matplotlib.pyplot as plt
import os

In [2]:
INIT_LR = 1e-4 # lirning rate 
EPOCHS = 20 # N. Ebochs 
BS = 32 # Bache size

In [3]:
DIRECTORY = r'D:\Me\Future\Ai courses\Data\masks' # path of my data 
CAEIGORIES = ['with_mask','without_mask'] # items im my path 

In [4]:
data = []
labels = []

# for loop to add images in data and add (with_mask or without_mask ) in labels

for category in CAEIGORIES:
    path = os.path.join(DIRECTORY,category)
    for img in os.listdir(path):
        img_path = os.path.join(path,img)
        image = load_img(img_path,target_size = (224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)

C:\Users\DR\anaconda3\lib\site-packages\PIL\Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
lb = LabelBinarizer() # to convert (with_mask or without_mask ) to 0,1

labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [6]:
# traning data and add X , Y

(X_train, X_test, y_train, y_test) = train_test_split(data, labels, test_size = 0.20, random_state = 42, stratify = labels)


In [7]:
# generator for data augmentation

aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [8]:
# create a new model by MobileNetV2 and ignore output   

baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

# create my spicial output by cnn

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [9]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [10]:
for layer in baseModel.layers:
	layer.trainable = False
    
# optimizer details     
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

C:\Users\DR\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
H = model.fit(
aug.flow(X_train, y_train, batch_size=BS),
steps_per_epoch=len(X_train) // BS,
validation_data=(X_test, y_test),
validation_steps=len(X_test) // BS,
epochs=EPOCHS)

Epoch 1/20
95/95 [==============================] - 71s 717ms/step - loss: 0.4097 - accuracy: 0.8573 - val_loss: 0.1522 - val_accuracy: 0.9817
Epoch 2/20
95/95 [==============================] - 64s 670ms/step - loss: 0.1492 - accuracy: 0.9651 - val_loss: 0.0753 - val_accuracy: 0.9935
Epoch 3/20
95/95 [==============================] - 61s 642ms/step - loss: 0.0983 - accuracy: 0.9759 - val_loss: 0.0553 - val_accuracy: 0.9922
Epoch 4/20
95/95 [==============================] - 61s 639ms/step - loss: 0.0823 - accuracy: 0.9769 - val_loss: 0.0501 - val_accuracy: 0.9909
Epoch 5/20
95/95 [==============================] - 60s 635ms/step - loss: 0.0650 - accuracy: 0.9829 - val_loss: 0.0416 - val_accuracy: 0.9922
Epoch 6/20
95/95 [==============================] - 60s 633ms/step - loss: 0.0583 - accuracy: 0.9829 - val_loss: 0.0364 - val_accuracy: 0.9935
Epoch 7/20
95/95 [==============================] - 61s 639ms/step - loss: 0.0506 - accuracy: 0.9865 - val_loss: 0.0342 - val_accuracy: 0.9935

In [13]:
# pridecting X_test
predIdxs = model.predict(X_test, batch_size=BS)

24/24 [==============================] - 12s 480ms/step


In [14]:
predIdxs = np.argmax(predIdxs, axis=1)

In [15]:
# save model 
model.save("mask_detector.model", save_format="h5")